# Fedbiomed Researcher base example

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [2]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

2022-01-20 11:06:33,226 fedbiomed INFO - Component environment:
2022-01-20 11:06:33,228 fedbiomed INFO - - type = ComponentType.RESEARCHER


Note : write **only** the code to export in the following cell

In [3]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpqr8n927r/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [4]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-01-20 11:10:29,841 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-01-20 11:10:29,846 fedbiomed INFO - log from: node_f290cd48-a70a-4e55-9262-81f802f9c95c / DEBUG - Message received: {'researcher_id': 'researcher_420cfc13-37cb-447c-af20-f7ac5cb2b6ab', 'tags': ['#MNIST', '#dataset'], 'command': 'search'}
2022-01-20 11:10:39,880 fedbiomed INFO - Node selected for training -> node_f290cd48-a70a-4e55-9262-81f802f9c95c


{'_tags': ['#MNIST', '#dataset'], '_nodes': None, '_reqs': <fedbiomed.researcher.requests.Requests object at 0x7f96a007c0a0>, '_fds': <fedbiomed.researcher.datasets.FederatedDataSet object at 0x7f95dd000c10>, '_aggregator': <fedbiomed.researcher.aggregators.fedavg.FedAverage object at 0x7f95dd000580>, '_node_selection_strategy': <fedbiomed.researcher.strategies.default_strategy.DefaultStrategy object at 0x7f95dd000e50>, '_round_init': 0, '_round_current': 0, '_experimentation_folder': 'Experiment_0002', '_model_class': None, '_model_path': '/home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpqr8n927r/class_export_mnist.py', '_model_args': {}, '_training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, '_rounds': 2}
{'_tags': ['#MNIST', '#dataset'], '_nodes': None, '_reqs': <fedbiomed.researcher.requests.Requests object at 0x7f96a007c0a0>, '_fds': <fedbiomed.researcher.datasets.FederatedDataSet object at 0x7f95dd000c10>, '_aggregator':

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [14]:
exp.run()

{'_tags': ['#MNIST', '#dataset'], '_nodes': None, '_reqs': <fedbiomed.researcher.requests.Requests object at 0x7f96a007c0a0>, '_fds': <fedbiomed.researcher.datasets.FederatedDataSet object at 0x7f95dd000c10>, '_aggregator': <fedbiomed.researcher.aggregators.fedavg.FedAverage object at 0x7f95dd000580>, '_node_selection_strategy': <fedbiomed.researcher.strategies.default_strategy.DefaultStrategy object at 0x7f95dd000e50>, '_round_init': 0, '_round_current': 0, '_experimentation_folder': 'Experiment_0002', '_model_class': None, '_model_path': '/home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpqr8n927r/class_export_mnist.py', '_model_args': {}, '_training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, '_rounds': 2, '_job': None, '_aggregated_params': {}, '_save_breakpoints': False, '_monitor': None}
{'_tags': ['#MNIST', '#dataset'], '_nodes': None, '_reqs': <fedbiomed.researcher.requests.Requests object at 0x7f96a007c0a0>, '_fds': <fe

Exception: Error while running the experiment: 

 	   FB413: Please create job with `.set_job()` before running the experiment

In [16]:
exp.set_job()

2022-01-20 11:13:58,134 fedbiomed ERROR - FB410: `model_class` is mandatory for setting `Job`.  Please initialize experiment with model class or use `.set_model_class()` method of the experiment


{'_tags': ['#MNIST', '#dataset'], '_nodes': None, '_reqs': <fedbiomed.researcher.requests.Requests object at 0x7f96a007c0a0>, '_fds': <fedbiomed.researcher.datasets.FederatedDataSet object at 0x7f95dd000c10>, '_aggregator': <fedbiomed.researcher.aggregators.fedavg.FedAverage object at 0x7f95dd000580>, '_node_selection_strategy': <fedbiomed.researcher.strategies.default_strategy.DefaultStrategy object at 0x7f95dd000e50>, '_round_init': 0, '_round_current': 0, '_experimentation_folder': 'Experiment_0002', '_model_class': None, '_model_path': '/home/scansiz/Desktop/Inria/development/fedbiomed/var/tmp/tmpqr8n927r/class_export_mnist.py', '_model_args': {}, '_training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, '_rounds': 2, '_job': None, '_aggregated_params': {}, '_save_breakpoints': False, '_monitor': None}
{'_tags': ['#MNIST', '#dataset'], '_nodes': None, '_reqs': <fedbiomed.researcher.requests.Requests object at 0x7f96a007c0a0>, '_fds': <fe

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe

Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D